In [259]:
import mysql.connector as msc
from mysql.connector import Error
import pandas as pd
import tkinter as tk
from tkinter import ttk, Canvas
from datetime import date

In [260]:
### SERVER CONNECTION FUNCTION ###

def server_connection(hostName, userName, userPassword, database):
    connection = None
    try:
        connection = msc.connect(
            host = hostName,
            user = userName,
            password = userPassword,
            db = database,
            autocommit = True
        )
        print("connection successful")
    except Error as error:
        print(f"Error: '{error}'")
    return connection  

In [261]:
### MAIN METHOD ###

frameList = []
accountEmail = []
currentDate = [date.today().strftime("%Y-%m-%d")]
#currentDate = [date(2021, 9, 26).strftime("%Y-%m-%d")]
flight_airline_dropdown = ["Delta Airlines", "Southwest Airlines", "American Airlines", "United Airlines", 
                                    "JetBlue Airways", "Spirit Airlines", "WestJet", "Interjet"]
state_dropdown = ["AL", "AK", "AZ", "AR", "CA", "CZ", "CO", "CT", "DC", "DE",
          "FL", "GA", "GU", "HI", "ID", "IL", "IN", "IA", "KS", "KY",
          "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE",
          "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR",
          "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA",
          "WV", "WI", "WY"]
          
time_zone_dropdown = ["EST", "CST", "MST", "PST"]
registerCondition = 0

# need to start sql first and enter your sql password into the third parameter
connection = server_connection("localhost", "root", "Passcode1!", "travel_reservation_service")
def main():
    root = tk.Tk()
    root.geometry("500x500")
    root.title("Login")
    root.resizable(height = False, width = False)

    loginSetUp(root)

    root.mainloop()

connection successful


In [262]:
### LOGIN ###

def login(root, email, password):
    print(email)
    
    condition = True

    cursor = connection.cursor()
    cursor.execute("select * from Accounts natural join Admins")
    for row in cursor.fetchall():
        if (email == row[0] and password == row[3]):
            accountEmail.append(email)
            destroyPage()
            adminHomeSetUp(root)
            condition = False
            break
    cursor.close()
    
    if (condition):
        cursor = connection.cursor()
        cursor.execute("select * from Accounts natural join Owners natural join Customer")
        for row in cursor.fetchall():
            if (email == row[0] and password == row[3]):
                accountEmail.append(email)
                destroyPage()
                ownerCustomerHomeSetUp(root)
                condition = False
                break
        cursor.close()
        
    if (condition):
        cursor = connection.cursor()
        cursor.execute("select * from Accounts natural join Owners")
        for row in cursor.fetchall():
            if (email == row[0] and password == row[3]):
                accountEmail.append(email)
                destroyPage()
                ownerHomeSetUp(root)
                condition = False
                break
        cursor.close()
    
    if (condition):
        cursor = connection.cursor()
        cursor.execute("select * from Accounts natural join Customer")
        for row in cursor.fetchall():
            if (email == row[0] and password == row[3]):
                accountEmail.append(email)
                destroyPage()
                customerHomeSetUp(root)
                condition = False
                break
        cursor.close()
    
def logout(root):
    if (len(accountEmail) != 0):
        del accountEmail[0]
    destroyPage()
    loginSetUp(root)

def backAdmin(root):
    destroyPage()
    adminHomeSetUp(root)

def backCustomer(root):
    destroyPage()
    customerHomeSetUp(root)
    
def backOwner(root):
    destroyPage()
    ownerHomeSetUp(root)

In [263]:
### ADMIN VIEWS ###

def viewAirlines(root, execution_string, sort_dict):
    root.title("View Airlines")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Airlines", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    fnameLabel = tk.Label(frame, text = "Name:", font = ("Comic Sans MS", 12))
    fnameLabel.place(x = 170, y = 100, anchor = tk.W)
    
    fnameEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fnameEntry.place(x = 220, y = 90)
    
    statement = "select * from view_airlines"
    complete_statement = statement;
    if (execution_string != None):
        statement = execution_string
        complete_statement = execution_string
    added = 0
    for sort in sort_dict:
        if len(sort_dict[sort]) > 0:
            if added == 1:
                complete_statement += ", " + sort_dict[sort]
            else:
                complete_statement += " order by " + sort_dict[sort]
                added = 1
    cursor = connection.cursor()
    cursor.execute(complete_statement)
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["Name", "Rating", "Total Flights", "Minimum Flight Cost"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 10)
    tree.place(y = 150)
    
    displayTable(frame, tree, df, columns, 120)
    
    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterAirlineViewButton(root, fnameEntry.get(), sort_dict))
    filterButton.place(x = 400, y = 100, anchor = tk.CENTER)
    
    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : resetAirlineViewButton(root, sort_dict))
    resetButton.place(x = 100, y = 100, anchor = tk.CENTER)
    
    sortRatingAscButton = tk.Button(frame, text = "Rating ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : airlineViewSort(root, 'rating', statement, sort_dict, 'asc'))
    sortRatingAscButton.place(x = 50, y = 450, anchor = tk.CENTER)
    
    sortRatingDescButton = tk.Button(frame, text = "Rating v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : airlineViewSort(root, 'rating', statement, sort_dict, 'desc'))
    sortRatingDescButton.place(x = 125, y = 450, anchor = tk.CENTER)
    
    sortTotalAscButton = tk.Button(frame, text = "Total ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : airlineViewSort(root, 'total_flights', statement, sort_dict, 'asc'))
    sortTotalAscButton.place(x = 200, y = 450, anchor = tk.CENTER)
    
    sortTotalDescButton = tk.Button(frame, text = "Total v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : airlineViewSort(root, 'total_flights', statement, sort_dict, 'desc'))
    sortTotalDescButton.place(x = 275, y = 450, anchor = tk.CENTER)
    
    sortCostAscButton = tk.Button(frame, text = "Cost ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : airlineViewSort(root, 'min_flight_cost', statement, sort_dict, 'asc'))
    sortCostAscButton.place(x = 350, y = 450, anchor = tk.CENTER)
    
    sortCostDescButton = tk.Button(frame, text = "Cost v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : airlineViewSort(root, 'min_flight_cost', statement, sort_dict, 'desc'))
    sortCostDescButton.place(x = 425, y = 450, anchor = tk.CENTER)
    
    frame.pack()
    
def filterAirlineViewButton(root, param, sort_dict):
    statement = "select * from view_airlines"
    if (len(param) > 0):
        statement += " where airline_name like '%" + param + "%'"
    viewAirlines(root, statement, sort_dict)
    
def resetAirlineViewButton(root, sort_dict):
    statement = "select * from view_airlines"
    viewAirlines(root, statement, {'rating': '', 'total_flights': '', 'min_flight_cost': ''})

def airlineViewSort(root, column, currentExecution, sort_dict, direction):
    if direction == "asc":
        sort_dict.update({column: "" + column + " ASC"})
    elif direction == "desc":
        sort_dict.update({column: "" + column + " DESC"})
    viewAirlines(root, currentExecution, sort_dict)

def viewAirports(root, execution_string, sort_dict):
    root.title("View Airports")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Airports", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)

    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : resetAirportViewButton(root, sort_dict))
    resetButton.place(x = 65, y = 20, anchor = tk.CENTER)
    
    statement = "select Airport_Id, Airport_Name, Time_Zone, concat(Street, ', ', City, ', ', State, ' ', Zip) as 'Address' from Airport"
    complete_statement = statement;

    if (execution_string != None):
        statement = execution_string
        complete_statement = statement
    added = 0
    for sort in sort_dict:
        if len(sort_dict[sort]) > 0:
            if added == 1:
                complete_statement += ", " + sort_dict[sort]
            else:
                complete_statement += " order by " + sort_dict[sort]
                added = 1
    cursor = connection.cursor()
    cursor.execute(complete_statement)
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["ID", "Name", "Time Zone", "Address"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 10)
    tree.place(y = 100)
    
    displayTable(frame, tree, df, columns, 120)

    idLabel = tk.Label(frame, text = "ID:", font = ("Comic Sans MS", 14))
    idLabel.place(x = 80, y = 370, anchor = tk.CENTER)

    idEntry = tk.Entry(frame, font = ("Comic Sans MS", 14), width = 10)
    idEntry.place(x = 100, y = 350)

    menuTextType = tk.StringVar()
    menuTextType.set("Null")
    timeZoneDrop = tk.OptionMenu(root, menuTextType, *time_zone_dropdown)
    timeZoneDrop.place(x = 300, y = 350)

    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterAirportsButton(root, idEntry.get(), menuTextType.get()))
    filterButton.place(x = 400, y = 370, anchor = tk.CENTER)


    sortIdAscButton = tk.Button(frame, text = "ID ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : airportViewSort(root, 'Airport_ID', statement, sort_dict, 'asc'))
    sortIdAscButton.place(x = 50, y = 450, anchor = tk.CENTER)
    
    sortIdDescButton = tk.Button(frame, text = "ID v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : airportViewSort(root, 'Airport_ID', statement, sort_dict, 'desc'))
    sortIdDescButton.place(x = 125, y = 450, anchor = tk.CENTER)
    
    sortNameAscButton = tk.Button(frame, text = "Name ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : airportViewSort(root, 'Airport_Name', statement, sort_dict, 'asc'))
    sortNameAscButton.place(x = 200, y = 450, anchor = tk.CENTER)
    
    sortNameDescButton = tk.Button(frame, text = "Name v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : airportViewSort(root, 'Airport_Name', statement, sort_dict, 'desc'))
    sortNameDescButton.place(x = 275, y = 450, anchor = tk.CENTER)
    
    sortTimeZoneAscButton = tk.Button(frame, text = "Time Zone ^", font = ("Comic Sans MS", 8), fg = "white", bg = "black", width = 10,
                          command = lambda : airportViewSort(root, 'Time_Zone', statement, sort_dict, 'asc'))
    sortTimeZoneAscButton.place(x = 350, y = 450, anchor = tk.CENTER)
    
    sortTimeZoneDescButton = tk.Button(frame, text = "Time Zone v", font = ("Comic Sans MS", 8), fg = "white", bg = "black", width = 10,
                          command = lambda : airportViewSort(root, 'Time_Zone', statement, sort_dict, 'desc'))
    sortTimeZoneDescButton.place(x = 430, y = 450, anchor = tk.CENTER)
    

    frame.pack()

def filterAirportsButton(root, id, tz):
    statement = "select Airport_Id, Airport_Name, Time_Zone, concat(Street, ', ', City, ', ', State, ' ', Zip) as 'Address' from Airport"
    hasFilter = 0
    if (tz != "Null" or len(id) != 0):
        hasFilter = 1
        statement += " where"
    
    if (hasFilter == 1):
        if (tz != "Null" and len(tz) > 0):
            statement += " time_zone = " + '"' + tz + '"'
            if (len(id) != 0):
                statement += " and airport_id like '%" + id + "%'"
                hasFilter = 2
        else:
            statement += " airport_id like '%" + id + "%'"
    
    viewAirports(root, statement, {'Airport_ID': '', 'Airport_Name': '', 'Time_Zone': ''}) 


def airportViewSort(root, column, currentExecution, sort_dict, direction):
    if direction == "asc":
        sort_dict.update({column: "" + column + " ASC"})
    elif direction == "desc":
        sort_dict.update({column: "" + column + " DESC"})
    viewAirports(root, currentExecution, sort_dict)

def resetAirportViewButton(root, sort_dict):
    statement = "select Airport_Id, Airport_Name, Time_Zone, concat(Street, ', ', City, ', ', State, ' ', Zip) as 'Address' from Airport"
    viewAirports(root, statement, {'Airport_ID': '', 'Airport_Name': '', 'Time_Zone': ''})

    
def viewCustomers(root, execution_string):
    root.title("View Customers")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Customers", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    fnameLabel = tk.Label(frame, text = "Name:", font = ("Comic Sans MS", 12))
    fnameLabel.place(x = 170, y = 100, anchor = tk.W)
    
    fnameEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fnameEntry.place(x = 220, y = 90)
    
    statement = "select * from view_customers"
    if (execution_string != None):
        statement = execution_string
    cursor = connection.cursor()
    print(statement)
    cursor.execute(statement)
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["Name", "Average Rating", "Location", "is Owner", "Total Seats Purchased"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 10)
    tree.place(y = 150)
    
    displayTable(frame, tree, df, columns, 96)
    
    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterCustViewButton(root, statement, fnameEntry.get()))
    filterButton.place(x = 400, y = 100, anchor = tk.CENTER)
    
    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : resetCustViewButton(root))
    resetButton.place(x = 100, y = 100, anchor = tk.CENTER)
    
    sortNameAscButton = tk.Button(frame, text = "Name ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 9,
                          command = lambda : sortCustomerView(root, statement, "customer_name", "asc"))
    sortNameAscButton.place(x = 100, y = 420, anchor = tk.CENTER)
    
    sortNameDescButton = tk.Button(frame, text = "Name v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 9,
                          command = lambda : sortCustomerView(root, statement, "customer_name", "desc"))
    sortNameDescButton.place(x = 200, y = 420, anchor = tk.CENTER)
    
    sortAvgAscButton = tk.Button(frame, text = "Avg Rating ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 9,
                          command = lambda : sortCustomerView(root, statement, "avg_rating", "asc"))
    sortAvgAscButton.place(x = 300, y = 420, anchor = tk.CENTER)
    
    sortAvgDescButton = tk.Button(frame, text = "Avg Rating v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 9,
                          command = lambda : sortCustomerView(root, statement, "avg_rating", "desc"))
    sortAvgDescButton.place(x = 400, y = 420, anchor = tk.CENTER)
    
    sortOwnerAscButton = tk.Button(frame, text = "Owner ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 9,
                          command = lambda : sortCustomerView(root, statement, "is_owner", "asc"))
    sortOwnerAscButton.place(x = 100, y = 460, anchor = tk.CENTER)
    
    sortOwnerDescButton = tk.Button(frame, text = "Owner v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 9,
                          command = lambda : sortCustomerView(root, statement, "is_owner", "desc"))
    sortOwnerDescButton.place(x = 200, y = 460, anchor = tk.CENTER)
    
    sortSeatsAscButton = tk.Button(frame, text = "Seats ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 9,
                          command = lambda : sortCustomerView(root, statement, "total_seats_purchased", "asc"))
    sortSeatsAscButton.place(x = 300, y = 460, anchor = tk.CENTER)
    
    sortSeatsDescButton = tk.Button(frame, text = "Seats v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 9,
                          command = lambda : sortCustomerView(root, statement, "total_seats_purchased", "desc"))
    sortSeatsDescButton.place(x = 400, y = 460, anchor = tk.CENTER)
    
    frame.pack()
    
def sortCustomerView(root, statement, param, direction):
    if (statement.find("order") != -1):
        statement = statement.split(" order")[0]
    statement += " order by " + param + " " + direction
    viewCustomers(root, statement)

def filterCustViewButton(root, statement, param):
    direction = ""
    if (statement.find("order") != -1):
        direction = " order" + statement.split("order")[1]
    statement = "select * from view_customers"
    if (len(param) > 0):
        statement += " where customer_name like '%" + param + "%'" + direction
    viewCustomers(root, statement)
    
def resetCustViewButton(root):
    statement = "select * from view_customers"
    viewCustomers(root, statement)
    
def viewOwners(root, execution_string, sort_dict):
    root.title("View Owners")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Owners", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    fnameLabel = tk.Label(frame, text = "Name:", font = ("Comic Sans MS", 12))
    fnameLabel.place(x = 170, y = 100, anchor = tk.W)
    
    fnameEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fnameEntry.place(x = 220, y = 90)
    
    statement = "select * from view_owners"
    complete_statement = statement;
    if (execution_string != None):
        statement = execution_string
        complete_statement = execution_string
    added = 0
    for sort in sort_dict:
        if len(sort_dict[sort]) > 0:
            if added == 1:
                complete_statement += ", " + sort_dict[sort]
            else:
                complete_statement += " order by " + sort_dict[sort]
                added = 1
    cursor = connection.cursor()
    cursor.execute(complete_statement)
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["Name", "Average Rating", "Number of Properties Owned", "Average Property Rating"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 12)
    tree.place(y = 120)
    
    displayTable(frame, tree, df, columns, 120)
    
    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterOwnersViewButton(root, fnameEntry.get(), sort_dict))
    filterButton.place(x = 400, y = 100, anchor = tk.CENTER)
    
    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : resetOwnersViewButton(root, sort_dict))
    resetButton.place(x = 100, y = 100, anchor = tk.CENTER)
    
    sortRatingAscButton = tk.Button(frame, text = "Name ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : ownersViewSort(root, 'owner_name', statement, sort_dict, 'asc'))
    sortRatingAscButton.place(x = 50, y = 450, anchor = tk.CENTER)
    
    sortRatingDescButton = tk.Button(frame, text = "Name v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : ownersViewSort(root, 'owner_name', statement, sort_dict, 'desc'))
    sortRatingDescButton.place(x = 125, y = 450, anchor = tk.CENTER)
    
    sortTotalAscButton = tk.Button(frame, text = "#Props ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : ownersViewSort(root, 'num_properties_owned', statement, sort_dict, 'asc'))
    sortTotalAscButton.place(x = 200, y = 450, anchor = tk.CENTER)
    
    sortTotalDescButton = tk.Button(frame, text = "#Props v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : ownersViewSort(root, 'num_properties_owned', statement, sort_dict, 'desc'))
    sortTotalDescButton.place(x = 275, y = 450, anchor = tk.CENTER)
    
    sortCostAscButton = tk.Button(frame, text = "Avg Rating ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : ownersViewSort(root, 'avg_property_rating', statement, sort_dict, 'asc'))
    sortCostAscButton.place(x = 350, y = 450, anchor = tk.CENTER)
    
    sortCostDescButton = tk.Button(frame, text = "Avg Rating v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : ownersViewSort(root, 'avg_property_rating', statement, sort_dict, 'desc'))
    sortCostDescButton.place(x = 425, y = 450, anchor = tk.CENTER)
    
    frame.pack()
    
def filterOwnersViewButton(root, param, sort_dict):
    statement = "select * from view_owners"
    if (len(param) > 0):
        statement += " where owner_name like '%" + param + "%'"
    viewOwners(root, statement, sort_dict)
    
def resetOwnersViewButton(root, sort_dict):
    statement = "select * from view_owners"
    viewOwners(root, statement, {'owner_name': '', 'num_properties_owned': '', 'avg_property_rating': ''})

def ownersViewSort(root, column, currentExecution, sort_dict, direction):
    if direction == "asc":
        sort_dict.update({column: "" + column + " ASC"})
    elif direction == "desc":
        sort_dict.update({column: "" + column + " DESC"})
    viewOwners(root, currentExecution, sort_dict)

In [264]:
### ADMIN PROCEDURES ###

def scheduleFlight(root):
    root.title("Schedule Flight")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    titleLabel = tk.Label(frame, text = "Schedule Flight", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    #left side entries
    
    fnumLabel = tk.Label(frame, text = "Flight Number:", font = ("Comic Sans MS", 12))
    fnumLabel.place(x = 0, y = 100, anchor = tk.W)
    
    fnumEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fnumEntry.place(x = 130, y = 90)
    
    airlineLabel = tk.Label(frame, text = "Airline:", font = ("Comic Sans MS", 12))
    airlineLabel.place(x = 0, y = 135, anchor = tk.W)
    
    menuTextType = tk.StringVar()
    menuTextType.set(flight_airline_dropdown[0])
    airlineDrop = tk.OptionMenu(root, menuTextType, *flight_airline_dropdown)
    airlineDrop.place(x = 125, y = 125)
    
    fromLabel = tk.Label(frame, text = "From Airport:", font = ("Comic Sans MS", 12))
    fromLabel.place(x = 0, y = 170, anchor = tk.W)
    
    fromEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fromEntry.place(x = 130, y = 160)
    
    toLabel = tk.Label(frame, text = "To Airport:", font = ("Comic Sans MS", 12))
    toLabel.place(x = 0, y = 205, anchor = tk.W)
    
    toEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    toEntry.place(x = 130, y = 195)
    
    depTimeLabel = tk.Label(frame, text = "Departure Time:", font = ("Comic Sans MS", 12))
    depTimeLabel.place(x = 0, y = 240, anchor = tk.W)
    
    depTimeEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    depTimeEntry.place(x = 130, y = 230)
    
    arrTimeLabel = tk.Label(frame, text = "Arrival Time:", font = ("Comic Sans MS", 12))
    arrTimeLabel.place(x = 0, y = 275, anchor = tk.W)
    
    arrTimeEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    arrTimeEntry.place(x = 130, y = 265)
    
    #right side entries
    
    fDateLabel = tk.Label(frame, text = "Flight Date:", font = ("Comic Sans MS", 12))
    fDateLabel.place(x = 250, y = 100, anchor = tk.W)
    
    fDateEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fDateEntry.place(x = 380, y = 90)
    
    costLabel = tk.Label(frame, text = "$ Per Person:", font = ("Comic Sans MS", 12))
    costLabel.place(x = 250, y = 135, anchor = tk.W)
    
    costEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    costEntry.place(x = 380, y = 125)
    
    capLabel = tk.Label(frame, text = "Capacity:", font = ("Comic Sans MS", 12))
    capLabel.place(x = 250, y = 170, anchor = tk.W)
    
    capEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    capEntry.place(x = 380, y = 160)
    
    d1 = currentDate[0]
    dateLabel = tk.Label(frame, text = "Current Date: " + d1, font = ("Comic Sans MS", 12))
    dateLabel.place(x = 250, y = 205, anchor = tk.W)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 400, y = 400, anchor = tk.CENTER)
    
    scheduleButton = tk.Button(frame, text = "Schedule", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : scheduleFlightButton(fnumEntry.get(), menuTextType.get(), fromEntry.get(), 
                                                                  toEntry.get(), depTimeEntry.get(), arrTimeEntry.get(), 
                                                                  fDateEntry.get(), costEntry.get(), capEntry.get(), d1))
    scheduleButton.place(x = 100, y = 400, anchor = tk.CENTER)
    
    frame.pack()

def scheduleFlightButton(Flight_Num, Airline_Name, From_Airport, To_Airport, Departure_Time, 
                         Arrival_Time, Flight_Date, Cost, Capacity, Current_Date):
    cursor = connection.cursor()
    cursor.callproc('schedule_flight', [Flight_Num, Airline_Name, From_Airport, To_Airport, Departure_Time, 
                                        Arrival_Time, Flight_Date, Cost, Capacity, Current_Date])
    cursor.close()
    
def removeFlight(root, importedFlightData, sort_dict):
    root.title("Remove Flight")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    titleLabel = tk.Label(frame, text = "Remove Flight", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    datesLabel = tk.Label(frame, text = "Dates:", font = ("Comic Sans MS", 12))
    datesLabel.place(x = 0, y = 100, anchor = tk.W)
    
    datesLeftEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    datesLeftEntry.place(x = 60, y = 90)
    
    dashLabel = tk.Label(frame, text = "-", font = ("Comic Sans MS", 12))
    dashLabel.place(x = 165, y = 100, anchor = tk.W)
    
    datesRightEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    datesRightEntry.place(x = 180, y = 90)
    
    airlineLabel = tk.Label(frame, text = "Airline:", font = ("Comic Sans MS", 12))
    airlineLabel.place(x = 290, y = 100, anchor = tk.W)
    
    menuTextType = tk.StringVar()
    menuTextType.set("Null")
    airlineDrop = tk.OptionMenu(root, menuTextType, *flight_airline_dropdown)
    airlineDrop.place(x = 360, y = 85)
    
    d1 = currentDate[0]
    dateLabel = tk.Label(frame, text = "Current Date: " + d1, font = ("Comic Sans MS", 12))
    dateLabel.place(x = 0, y = 135, anchor = tk.W)
    
    fnumLabel = tk.Label(frame, text = "Flight Number:", font = ("Comic Sans MS", 12))
    fnumLabel.place(x = 230, y = 135, anchor = tk.W)
    
    fnumEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fnumEntry.place(x = 350, y = 125)
    
    statement = "select airline_name, flight_num, flight_date from Flight"
    complete_statement = statement;
    if (importedFlightData != None):
        statement = importedFlightData
        complete_statement = importedFlightData
    added = 0
    for sort in sort_dict:
        if len(sort_dict[sort]) > 0:
            if added == 1:
                complete_statement += ", " + sort_dict[sort]
            else:
                complete_statement += " order by " + sort_dict[sort]
                added = 1
    cursor = connection.cursor()
    cursor.execute(complete_statement)
    
    flightData = []
    
    for row in cursor.fetchall():
        flightData.append(row)
    cursor.close()
    
    columns = ["Airline", "Number", "Date"]
    df = pd.DataFrame(flightData, columns = columns)
    
    tree = ttk.Treeview(frame, height = 10)
    tree.place(y = 165)

    displayTable(frame, tree, df, columns, 96)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 50, y = 450, anchor = tk.CENTER)
    
    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : resetFlightsButton(statement, root))
    resetButton.place(x = 200, y = 450, anchor = tk.CENTER)
    
    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterFlightsButton(statement, datesLeftEntry.get(), datesRightEntry.get(), 
                                                                 menuTextType.get(), fnumEntry.get(), tree, frame, root, sort_dict))
    filterButton.place(x = 300, y = 450, anchor = tk.CENTER)
    
    removeButton = tk.Button(frame, text = "Remove", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : removeFlightButton(flightData, tree, tree.selection(), d1, root, sort_dict))
    removeButton.place(x = 450, y = 450, anchor = tk.CENTER)
    
    sortAirlineAscButton = tk.Button(frame, text = "Airline ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : removeFlightSort(root, 'airline_name', statement, sort_dict, 'asc'))
    sortAirlineAscButton.place(x = 370, y = 180, anchor = tk.CENTER)
    
    sortAirlineDescButton = tk.Button(frame, text = "Airline v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : removeFlightSort(root, 'airline_name', statement, sort_dict, 'desc'))
    sortAirlineDescButton.place(x = 450, y = 180, anchor = tk.CENTER)
    
    sortFnumAscButton = tk.Button(frame, text = "Flight# ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : removeFlightSort(root, 'flight_num', statement, sort_dict, 'asc'))
    sortFnumAscButton.place(x = 370, y = 220, anchor = tk.CENTER)
    
    sortFnumDescButton = tk.Button(frame, text = "Flight# v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : removeFlightSort(root, 'flight_num', statement, sort_dict, 'desc'))
    sortFnumDescButton.place(x = 450, y = 220, anchor = tk.CENTER)
    
    sortDateAscButton = tk.Button(frame, text = "Date ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : removeFlightSort(root, 'flight_date', statement, sort_dict, 'asc'))
    sortDateAscButton.place(x = 370, y = 260, anchor = tk.CENTER)
    
    sortDateDescButton = tk.Button(frame, text = "Date v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : removeFlightSort(root, 'flight_date', statement, sort_dict, 'desc'))
    sortDateDescButton.place(x = 450, y = 260, anchor = tk.CENTER)
    
    frame.pack()
    
def resetFlightsButton(flightData, root):
    removeFlight(root, None, {'airline_name': '', 'flight_num': '', 'flight_date': ''})

def filterFlightsButton(flightData, startDate, endDate, airline, num, tree, frame, root, sort_dict):
    cursor = connection.cursor()
    execution_string = "select airline_name, flight_num, flight_date from Flight"
    filterExists = 0;
    if (len(startDate) == 10 or len(endDate) == 10 or airline != "Null" or len(num) > 0):
        filterExists = 1;
        execution_string += " where "
    if (filterExists == 1):
        if (len(startDate) == 10):
            if (len(endDate) == 10):
                execution_string = execution_string + "flight_date between '" + startDate + "'' and ''" + endDate + "'"
                filterExists = 2
            else:
                execution_string = execution_string + "flight_date >= '" + startDate + "'"
                filterExists = 2
        if (len(endDate) == 10 and filterExists == 1):
            execution_string = execution_string + "flight_date <= '" + endDate + "'"
            filterExists = 2
        if (airline != "Null"):
            if (filterExists == 2):
                execution_string += " and "
            execution_string = execution_string + "airline_name = " + '"' + airline + '"'
            filterExists = 3
        if (len(num) > 0):
            if (filterExists == 2 or filterExists == 3):
                execution_string += " and "
            execution_string = execution_string + "flight_num = " + num
    removeFlight(root, execution_string, sort_dict)

def removeFlightButton(flightData, tree, treeSelection, currDate, root, sort_dict):
    num = tree.item(treeSelection)["values"][1]
    name = tree.item(treeSelection)["values"][0]
    cursor = connection.cursor()
    cursor.callproc('remove_flight', [num, name, currDate])
    cursor.close()
    removeFlight(root, None, sort_dict)
    
def removeFlightSort(root, column, currentExecution, sort_dict, direction):
    if direction == "asc":
        sort_dict.update({column: "" + column + " ASC"})
    elif direction == "desc":
        sort_dict.update({column: "" + column + " DESC"})
    removeFlight(root, currentExecution, sort_dict)

def processDate(root):
    root.title("Process Date")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    titleLabel = tk.Label(frame, text = "Process Date", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    setLabel = tk.Label(frame, text = "Set Current\nSystem Date:", font = ("Comic Sans MS", 14))
    setLabel.place(x = 170, y = 170, anchor = tk.E)
    
    entry = tk.Entry(frame, font = ("Comic Sans MS", 14), width = 20)
    entry.place(x = 180, y = 155)
    
    setButton = tk.Button(frame, text = "Set Date", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : setDateButton(entry.get()))
    setButton.place(x = 390, y = 210, anchor = tk.CENTER)
    
    frame.pack()

def setDateButton(date):
    cursor = connection.cursor()
    cursor.callproc('process_date', [date])
    del currentDate[0]
    currentDate.append(date)
    cursor.close()

In [265]:
### CUSTOMER VIEWS ###

def viewProperties(root, execution_string):
    root.title("View Properties")
    destroyPage()

    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Properties", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)

    minCapLabel = tk.Label(frame, text = "Min: ", font = ("Comic Sans MS", 12))
    minCapLabel.place(x = 150, y = 100, anchor = tk.W)

    minCapEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 5)
    minCapEntry.place(x = 190, y = 90)

    maxCapLabel = tk.Label(frame, text = "Max: ", font = ("Comic Sans MS", 12))
    maxCapLabel.place(x = 250, y = 100, anchor = tk.W)

    maxCapEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 5)
    maxCapEntry.place(x = 290, y = 90)

    statement = "select * from view_properties"
    if (execution_string != None):
        statement = execution_string
    cursor = connection.cursor()
    print(statement)
    cursor.execute(statement)

    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()

    columns = ["Name", "Average Rating", "Description", "Address", "Capacity", "Cost"]
    df = pd.DataFrame(data, columns = columns)

    tree = ttk.Treeview(frame, height = 12)
    tree.place(y = 150)
    
    displayTable(frame, tree, df, columns, 80)
    
    sortCostAscButton = tk.Button(frame, text = "Cost ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : propertyViewSort(root, statement, "asc"))
    sortCostAscButton.place(x = 187.5, y = 460, anchor = tk.CENTER)
    
    sortCostDescButton = tk.Button(frame, text = "Cost v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : propertyViewSort(root, statement, "desc"))
    sortCostDescButton.place(x = 312.5, y = 460, anchor = tk.CENTER)
    
    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7, 
                            command = lambda : filterPropertyViewButton(root, statement, minCapEntry.get(), maxCapEntry.get()))
    filterButton.place(x = 400, y = 100, anchor = tk.CENTER)
    
    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : resetPropertyViewButton(root))
    resetButton.place(x = 100, y = 100, anchor = tk.CENTER)

    frame.pack()
    
def propertyViewSort(root, statement, direction):
    if (statement.find("order") != 1):
        statement = statement.split(" order")[0]
    statement += " order by cost_per_night " + direction
    viewProperties(root, statement)

def filterPropertyViewButton(root, statement, minCap, maxCap):
    direction = ""
    if (statement.find("order") != -1):
        direction = " order" + statement.split("order")[1]
    statement = "select * from view_properties "
    if (minCap != "" and maxCap != ""):
        statement += "where capacity between " + minCap + " and " + maxCap
    
    elif (minCap != ""):
        statement += "where capacity >= " + minCap

    elif (maxCap != ""):
        statement += "where capacity <= " + maxCap
    statement += direction

    viewProperties(root, statement)

def resetPropertyViewButton(root):
    statement = "select * from view_properties"
    viewProperties(root, statement)

def viewReservations(root, inputData):
    root.title("View Property Reservations")
    destroyPage()

    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Reservations", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)

    ownerLabel = tk.Label(frame, text = "Owner Email:", font = ("Comic Sans MS", 12))
    ownerLabel.place(x = 50, y = 100, anchor = tk.W)

    ownerEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    ownerEntry.place(x = 150, y = 90)

    propertyLabel = tk.Label(frame, text = "Property:", font = ("Comic Sans MS", 12))
    propertyLabel.place(x = 250, y = 100, anchor = tk.W)

    propertyEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    propertyEntry.place(x = 325, y = 90)

    if inputData != None:
        data = inputData

        columns = ["Property", "Start Date", "End Date", "Cust. Phone", "Customer", "Cost", "Review", "Rating"]
        df = pd.DataFrame(data, columns = columns)

        tree = ttk.Treeview(frame, height = 12)
        tree.place(y = 150)
        
        displayTable(frame, tree, df, columns, 60)

    submitButton = tk.Button(frame, text = "Submit", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : viewReservationsButton(root, ownerEntry.get(), propertyEntry.get()))
    submitButton.place(x = 350, y = 475, anchor = tk.CENTER)

    frame.pack()

def viewReservationsButton(root, owner, property):
    if owner != None and owner != "" and property != None and property != "":
        cursor = connection.cursor()
        cursor.callproc('view_individual_property_reservations', [property, owner])
        cursor.execute('select * from view_individual_property_reservations')

        data = []
        for row in cursor.fetchall():
            data.append(row)
        cursor.close()
        viewReservations(root, data)
    else: 
        viewReservations(root, None)


In [266]:
### CUSTOMER PROCEDURES ###

def bookFlight(root, execution_string, sort_dict):
    root.title("Book Flight")
    frameList[0].pack_forget()
    del frameList[0]
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "Book Flight", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    statement = "select airline_name, flight_num, from_airport, to_airport, calc_seats_remaining(flight_num, airline_name), flight_date from flight"
    complete_statement = statement

    if (execution_string != None):
        statement = execution_string
        complete_statement = statement
    added = 0
    for sort in sort_dict:
        if len(sort_dict[sort]) > 0:
            if added == 1:
                complete_statement += ", " + sort_dict[sort]
            else:
                complete_statement += " order by " + sort_dict[sort]
                added = 1
    cursor = connection.cursor()
    cursor.execute(complete_statement)
    

    flightData = []
    for row in cursor.fetchall():
        flightData.append(row)
    selectStatement = "select book.airline_name, book.flight_num, num_seats * cost from book, flight where book.flight_num = flight.flight_num and book.airline_name = flight.airline_name and book.customer = %(email)s and book.was_cancelled = 0"
    cursor.execute(selectStatement, { 'email': accountEmail[0] })
    bookingData = []
    for row in cursor.fetchall():
            bookingData.append(row)
    cursor.close()
    
    columns = ["Airline", "Flight Num", "From", "To", "Avail. Seats", "Date"]
    df = pd.DataFrame(flightData, columns = columns)
    
    tree = ttk.Treeview(frame, height = 9)
    tree.place(y = 100)

    displayTableScroll(frame, tree, df, columns, [118, 70, 50, 50, 110, 79], 100, 293)

    seatLabel = tk.Label(frame, text = "Seats to Book:", font = ("Comic Sans MS", 12))
    seatLabel.place(x = 0, y = 320)

    seatEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    seatEntry.place(x = 110, y = 320)

    bookButton = tk.Button(frame, text = "Book", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : bookFlightButton(root, tree, tree.selection(), seatEntry.get()))
    bookButton.place(x = 220, y = 315)

    bookedLabel = tk.Label(frame, text = "Booked Flights:", font = ("Comic Sans MS", 14))
    bookedLabel.place(x = 250, y = 370, anchor = tk.CENTER)

    columnsTwo = ["Airline", "Flight Number", "Amount Spent"]
    dfTwo = pd.DataFrame(bookingData, columns = columnsTwo)
    
    treeTwo = ttk.Treeview(frame, height = 4)
    treeTwo.place(y = 393)

    displayTableScroll(frame, treeTwo, dfTwo, columnsTwo, 160, 393, 107)
    
    sortAirlineAscButton = tk.Button(frame, text = "Airline ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                command = lambda : bookFlightSort(root, 'Airline_Name', statement, sort_dict, 'asc'))
    sortAirlineAscButton.place(x = 370, y = 330, anchor = tk.CENTER)
    
    sortAirlineDescButton = tk.Button(frame, text = "Airline v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                              command = lambda : bookFlightSort(root, 'Airline_Name', statement, sort_dict, 'desc'))
    sortAirlineDescButton.place(x = 440, y = 330, anchor = tk.CENTER)
    
    sortIDAscButton = tk.Button(frame, text = "Num ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : bookFlightSort(root, 'Flight_Num', statement, sort_dict, 'asc'))
    sortIDAscButton.place(x = 370, y = 365, anchor = tk.CENTER)
    
    sortIDDescButton = tk.Button(frame, text = "Num v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : bookFlightSort(root, 'Flight_Num', statement, sort_dict, 'desc'))
    sortIDDescButton.place(x = 440, y = 365, anchor = tk.CENTER)

    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : resetBookFlights(root))
    resetButton.place(x = 30, y = 20, anchor = tk.CENTER)

    frame.pack()


# if something goes wrong with flight_num sort, remove +0 and uncomment addt'l dict_update
def bookFlightSort(root, column, currentExecution, sort_dict, direction):
    if (column == 'Flight_Num'):
        if direction == "asc":
            sort_dict.update({column: "" + column + "+0 ASC"})
            #sort_dict.update({column: "LENGTH(" + column + ")"})
        elif direction == "desc":
            sort_dict.update({column: "" + column + "+0 DESC"})
            #sort_dict.update({column: "LENGTH(" + column + ")"})

    else:
        if direction == "asc":
            sort_dict.update({column: "" + column + " ASC"})
        elif direction == "desc":
            sort_dict.update({column: "" + column + " DESC"})
    bookFlight(root, currentExecution, sort_dict)

def resetBookFlights(root):
    statement = "select airline_name, flight_num, from_airport, to_airport, calc_seats_remaining(flight_num, airline_name), flight_date from flight"
    bookFlight(root, statement, {'Airline_Name': '', 'Flight_Num': ''})

def bookFlightButton(root, tree, treeSelection, seats):
    d1 = currentDate[0]
    if seats != "" and int(seats) > 0:
        cursor = connection.cursor()
        for ind in treeSelection:
            row = tree.item(tree.get_children()[int(ind)])['values']
            cursor.callproc('book_flight', [accountEmail[0], row[1], row[0], seats, d1])
        cursor.close()
        bookFlight(root)

def cancelBookedFlight(root, importedFlightData):
    root.title("Cancel Flight")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    titleLabel = tk.Label(frame, text = "Cancel Flight", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    airlineLabel = tk.Label(frame, text = "Airline:", font = ("Comic Sans MS", 12))
    airlineLabel.place(x = 290, y = 100, anchor = tk.W)
    
    menuTextType = tk.StringVar()
    menuTextType.set("Null")
    airlineDrop = tk.OptionMenu(root, menuTextType, *flight_airline_dropdown)
    airlineDrop.place(x = 360, y = 85)
    
    d1 = currentDate[0]
    dateLabel = tk.Label(frame, text = "Current Date: " + d1, font = ("Comic Sans MS", 12))
    dateLabel.place(x = 0, y = 135, anchor = tk.W)
    
    fnumLabel = tk.Label(frame, text = "Flight Number:", font = ("Comic Sans MS", 12))
    fnumLabel.place(x = 230, y = 135, anchor = tk.W)
    
    fnumEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fnumEntry.place(x = 350, y = 125)
    
    flightData = []
    if (importedFlightData != None):
        flightData = importedFlightData
    else: 
        cursor = connection.cursor()
        selectStatement = "select book.airline_name, book.flight_num, flight.flight_date from book, flight where book.airline_name = flight.airline_name and book.flight_num = flight.flight_num and customer = %(email)s and was_cancelled = 0"
        cursor.execute(selectStatement, { 'email': accountEmail[0] })
        for row in cursor.fetchall():
            flightData.append(row)
        cursor.close()
    
    columns = ["Airline", "Number", "Date"]
    df = pd.DataFrame(flightData, columns = columns)
    
    tree = ttk.Treeview(frame, height = 10)
    tree.place(y = 165)

    displayTable(frame, tree, df, columns, 96)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 50, y = 450, anchor = tk.CENTER)
    
    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : cancelBookedFlight(root, None))
    resetButton.place(x = 200, y = 450, anchor = tk.CENTER)
    
    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterBookingsButton(flightData, menuTextType.get(), fnumEntry.get(), tree, frame, root))
    filterButton.place(x = 300, y = 450, anchor = tk.CENTER)
    
    removeButton = tk.Button(frame, text = "Cancel", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : removeBookingButton(tree, tree.selection(), d1, root))
    removeButton.place(x = 450, y = 450, anchor = tk.CENTER)

    sortAirlineAscButton = tk.Button(frame, text = "Airline ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                command = lambda : bookFlightSort(root, 'Airline_Name', statement, sort_dict, 'asc'))
    sortAirlineAscButton.place(x = 370, y = 330, anchor = tk.CENTER)
    
    sortAirlineDescButton = tk.Button(frame, text = "Airline v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                              command = lambda : bookFlightSort(root, 'Airline_Name', statement, sort_dict, 'desc'))
    sortAirlineDescButton.place(x = 440, y = 330, anchor = tk.CENTER)
    
    sortIDAscButton = tk.Button(frame, text = "Num ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : bookFlightSort(root, 'Flight_Num', statement, sort_dict, 'asc'))
    sortIDAscButton.place(x = 370, y = 365, anchor = tk.CENTER)
    
    sortIDDescButton = tk.Button(frame, text = "Num v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : bookFlightSort(root, 'Flight_Num', statement, sort_dict, 'desc'))
    sortIDDescButton.place(x = 440, y = 365, anchor = tk.CENTER)

    sortDateAscButton = tk.Button(frame, text = "Num ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : bookFlightSort(root, 'Flight_Num', statement, sort_dict, 'asc'))
    sortDateAscButton.place(x = 370, y = 365, anchor = tk.CENTER)
    
    sortDateDescButton = tk.Button(frame, text = "Num v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : bookFlightSort(root, 'Flight_Num', statement, sort_dict, 'desc'))
    sortDateDescButton.place(x = 440, y = 365, anchor = tk.CENTER)

    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : resetBookFlights(root))
    resetButton.place(x = 30, y = 20, anchor = tk.CENTER)
    
    frame.pack()

def filterBookingsButton(flightData, airline, num, tree, frame, root):
    flightData.clear()
    cursor = connection.cursor()
    execution_string = "select book.airline_name, book.flight_num, flight.flight_date from book, flight where book.airline_name = flight.airline_name and book.flight_num = flight.flight_num and customer = %(email)s and was_cancelled = 0"
    if (airline != "Null"):
        execution_string += " and "
        execution_string = execution_string + "book.airline_name = " + '"' + airline + '"'
    if (len(num) > 0):
        execution_string += " and "
        execution_string = execution_string + "book.flight_num = " + num

    cursor.execute(execution_string, { 'email': accountEmail[0] })
    for row in cursor.fetchall():
        flightData.append(row)
    cursor.close()
    cancelBookedFlight(root, flightData)

def removeBookingButton(tree, treeSelection, currDate, root):
    cursor = connection.cursor()
    for ind in treeSelection:
        row = tree.item(tree.get_children()[int(ind)])['values']
        num = row[1]
        name = row[0]
        cursor.callproc('cancel_flight_booking', [accountEmail[0], num, name, currDate])
    cursor.close()
    cancelBookedFlight(root, None)

def reserveProperty(root, execution_string):
    root.title("Reserve Property")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    titleLabel = tk.Label(frame, text = "Reserve Property", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 30, anchor = tk.CENTER)
    
    datesLabel = tk.Label(frame, text = "Dates:", font = ("Comic Sans MS", 12))
    datesLabel.place(x = 10, y = 70, anchor = tk.W)
    
    startDateEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    startDateEntry.place(x = 60, y = 55)
    
    hyphenLabel = tk.Label(frame, text = "-", font = ("Comic Sans MS", 12))
    hyphenLabel.place(x = 165, y = 70, anchor = tk.W)
    
    endDateEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    endDateEntry.place(x = 180, y = 55)
    
    d1 = currentDate[0]
    dateLabel = tk.Label(frame, text = "Current Date: " + d1, font = ("Comic Sans MS", 12))
    dateLabel.place(x = 300, y = 70, anchor = tk.W)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : reserveProperty(root, None))
    resetButton.place(x = 35, y = 20, anchor = tk.CENTER)
    
    statement = "select property_name, owner_email, capacity from property"
    if (execution_string != None):
        statement = execution_string
    cursor = connection.cursor()
    print(statement)
    cursor.execute(statement)
    
    propertyData = []
    for row in cursor.fetchall():
        propertyData.append(row)
    selectStatement = "select r.property_name, r.start_date, r.end_date, p.cost * (r.end_date - r.start_date) from reserve r, property p where r.property_name = p.property_name and r.owner_email = p.owner_email and customer = %(email)s and r.was_cancelled = 0"
    cursor.execute(selectStatement, { 'email': accountEmail[0] })
    reservationData = []
    for row in cursor.fetchall():
            reservationData.append(row)
    cursor.close()
    
    columns = ["Property Name", "Owner Email", "Capacity"]
    df = pd.DataFrame(propertyData, columns = columns)
    
    tree = ttk.Treeview(frame, height = 7)
    tree.place(y = 100)

    displayTableScroll(frame, tree, df, columns, 160, 100, 293)
    
    guestsLabel = tk.Label(frame, text = "Number of Guests:", font = ("Comic Sans MS", 12))
    guestsLabel.place(x = 0, y = 310)

    guestsEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    guestsEntry.place(x = 145, y = 310)

    reserveButton = tk.Button(frame, text = "Reserve", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : reservePropertyButton(root, tree, tree.selection(), startDateEntry.get(),
                                                                   endDateEntry.get(), guestsEntry.get(), d1, statement))
    reserveButton.place(x = 255, y = 310)

    bookedLabel = tk.Label(frame, text = "Reserved Properties:", font = ("Comic Sans MS", 14))
    bookedLabel.place(x = 250, y = 380, anchor = tk.CENTER)

    columnsTwo = ["Property Name", "Start Date", "End Date", "Amount Spent"]
    dfTwo = pd.DataFrame(reservationData, columns = columnsTwo)
    
    treeTwo = ttk.Treeview(frame, height = 4)
    treeTwo.place(y = 393)

    displayTableScroll(frame, treeTwo, dfTwo, columnsTwo, 120, 393, 107)
    
    sortNameAscButton = tk.Button(frame, text = "Name ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                 command = lambda : reservePropertyFilter(root, "property_name", "asc"))
    sortNameAscButton.place(x = 370, y = 310, anchor = tk.CENTER)
    
    sortNameDescButton = tk.Button(frame, text = "Name v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                  command = lambda : reservePropertyFilter(root, "property_name", "desc"))
    sortNameDescButton.place(x = 440, y = 310, anchor = tk.CENTER)
    
    sortCapacityAscButton = tk.Button(frame, text = "Capacity ^", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                     command = lambda : reservePropertyFilter(root, "capacity", "asc"))
    sortCapacityAscButton.place(x = 370, y = 345, anchor = tk.CENTER)
    
    sortCapacityDescButton = tk.Button(frame, text = "Capcity v", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                      command = lambda : reservePropertyFilter(root, "capacity", "desc"))
    sortCapacityDescButton.place(x = 440, y = 345, anchor = tk.CENTER)
    
    frame.pack()

def reservePropertyFilter(root, param, direction):
    statement = "select property_name, owner_email, capacity from property order by " + param + " " + direction
    reserveProperty(root, statement)

def reservePropertyButton(root, tree, treeSelection, startDate, endDate, numGuests, currDate, statement):
    cursor = connection.cursor()
    for ind in treeSelection:
        row = tree.item(tree.get_children()[int(ind)])['values']
        owner = row[1]
        name = row[0]
        cursor.callproc('reserve_property', [name, owner, accountEmail[0], startDate, endDate, numGuests, currDate])
    cursor.close()
    reserveProperty(root, statement)

def cancelReservation(root):
    root.title("Cancel Property")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "Cancel Property", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    cursor = connection.cursor()
    selectStatement = "select r.start_date, r.end_date, r.property_name, r.owner_email, concat(p.street, ', ', p.city, ', ', p.state, ' ', p.zip) from reserve r, property p where r.property_name = p.property_name and r.owner_email = p.owner_email and r.customer = %(email)s and was_cancelled = 0"
    cursor.execute(selectStatement, { 'email': accountEmail[0] })
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["Start Date", "End Date", "Property Name", "Owner Email", "Address"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 12)
    tree.place(y = 100)
    
    displayTable(frame, tree, df, columns, 96)
    
    cancelButton = tk.Button(frame, text = "Cancel", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : cancelReservationButton(root, tree, tree.selection()))
    cancelButton.place(x = 450, y = 450, anchor = tk.CENTER)

    frame.pack()

def cancelReservationButton(root, tree, treeSelection):
    cursor = connection.cursor()
    for ind in treeSelection:
        row = tree.item(tree.get_children()[int(ind)])['values']
        owner = row[3]
        name = row[2]
        cursor.callproc('cancel_property_reservation', [name, owner, accountEmail[0], currentDate[0]])
    cursor.close()
    cancelReservation(root)

def reviewProperty(root):
    root.title("Review Property")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "Review Property", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    cursor = connection.cursor()
    selectStatement = "select r.start_date, r.end_date, r.property_name, r.owner_email, concat(p.street, ', ', p.city, ', ', p.state, ' ', p.zip) from reserve r join property p on r.property_name = p.property_name and r.owner_email = p.owner_email left join review v on r.owner_email = v.owner_email and r.property_name = v.property_name where r.customer = %(email)s and r.was_cancelled = 0 and v.property_name is null"
    cursor.execute(selectStatement, { 'email': accountEmail[0] })
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["Start Date", "End Date", "Property Name", "Owner Email", "Address"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 12)
    tree.place(y = 100)
    
    displayTable(frame, tree, df, columns, 96)

    messageEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 40)
    messageEntry.place(x = 50, y = 425)

    scoreLabel = tk.Label(frame, text = "Score:", font = ("Comic Sans MS", 12))
    scoreLabel.place(x = 125, y = 475, anchor = tk.CENTER)

    scoreEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    scoreEntry.place(x = 150, y = 460)
    
    submitButton = tk.Button(frame, text = "Submit", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : submitReviewButton(root, tree, tree.selection(), messageEntry.get(), scoreEntry.get()))
    submitButton.place(x = 350, y = 475, anchor = tk.CENTER)

    frame.pack()

def submitReviewButton(root, tree, treeSelection, message, score):
    if message != None and message != "" and score != None and score != "" and int(score) >= 0 and int(score) <= 5:
        cursor = connection.cursor()
        for ind in treeSelection:
            row = tree.item(tree.get_children()[int(ind)])['values']
            owner = row[3]
            name = row[2]
            cursor.callproc('customer_review_property', [name, owner, accountEmail[0], message, int(score), currentDate[0]])
        cursor.close()
        reviewProperty(root)

def rateOwner(root):
    root.title("Rate Owner")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "Rate Owner", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    cursor = connection.cursor()
    selectStatement = "select r.start_date, r.end_date, r.owner_email, r.property_name, concat(p.street, ', ', p.city, ', ', p.state, ' ', p.zip) from reserve r join property p on r.property_name = p.property_name and r.owner_email = p.owner_email left join customers_rate_owners c on r.owner_email = c.owner_email where r.customer = %(email)s and r.was_cancelled = 0 and c.owner_email is null"
    cursor.execute(selectStatement, { 'email': accountEmail[0] })
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["Start Date", "End Date", "Owner Email", "Property Name", "Address"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 12, selectmode="browse")
    tree.place(y = 100)
    
    displayTable(frame, tree, df, columns, 96)

    scoreLabel = tk.Label(frame, text = "Rating:", font = ("Comic Sans MS", 12))
    scoreLabel.place(x = 125, y = 475, anchor = tk.CENTER)

    scoreEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    scoreEntry.place(x = 165, y = 460)
    
    submitButton = tk.Button(frame, text = "Submit", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : submitRatingButton(root, tree, tree.selection(), scoreEntry.get()))
    submitButton.place(x = 350, y = 475, anchor = tk.CENTER)

    frame.pack()

def submitRatingButton(root, tree, treeSelection, score):
    if score != None and score != "" and int(score) >= 0 and int(score) <= 5:
        cursor = connection.cursor()
        for ind in treeSelection:
            row = tree.item(tree.get_children()[int(ind)])['values']
            owner = row[2]
            cursor.callproc('customer_rates_owner', [accountEmail[0], owner, int(score), currentDate[0]])
        cursor.close()
        rateOwner(root)

In [267]:
### OWNER ###

def addProperty(root):
    root.title("Add Property")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    titleLabel = tk.Label(frame, text = "Add Property", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    #left side entries
    
    nameLabel = tk.Label(frame, text = "Name:", font = ("Comic Sans MS", 12))
    nameLabel.place(x = 0, y = 100, anchor = tk.W)
    
    nameEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    nameEntry.place(x = 130, y = 90)
    
    streetLabel = tk.Label(frame, text = "Street:", font = ("Comic Sans MS", 12))
    streetLabel.place(x = 0, y = 135, anchor = tk.W)
    
    streetEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    streetEntry.place(x = 130, y = 125)
    
    cityLabel = tk.Label(frame, text = "City:", font = ("Comic Sans MS", 12))
    cityLabel.place(x = 0, y = 170, anchor = tk.W)
    
    cityEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    cityEntry.place(x = 130, y = 160)
    
    stateLabel = tk.Label(frame, text = "State:", font = ("Comic Sans MS", 12))
    stateLabel.place(x = 0, y = 205, anchor = tk.W)
    
    menuTextType = tk.StringVar()
    menuTextType.set(state_dropdown[0])
    stateDrop = tk.OptionMenu(root, menuTextType, *state_dropdown)
    stateDrop.place(x = 125, y = 195)
    
    zipLabel = tk.Label(frame, text = "Zip:", font = ("Comic Sans MS", 12))
    zipLabel.place(x = 0, y = 240, anchor = tk.W)
    
    zipEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    zipEntry.place(x = 130, y = 230)
    
    nearLabel = tk.Label(frame, text = "Nearest Airport:", font = ("Comic Sans MS", 12))
    nearLabel.place(x = 0, y = 275, anchor = tk.W)
    
    nearEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    nearEntry.place(x = 130, y = 265)
    
    distanceLabel = tk.Label(frame, text = "Dis to Airport:", font = ("Comic Sans MS", 12))
    distanceLabel.place(x = 0, y = 310, anchor = tk.W)
    
    distanceEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    distanceEntry.place(x = 130, y = 300)
    
    capacityLabel = tk.Label(frame, text = "Capacity:", font = ("Comic Sans MS", 12))
    capacityLabel.place(x = 0, y = 345, anchor = tk.W)
    
    capacityEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    capacityEntry.place(x = 130, y = 335)
    
    #right side entries
    
    descriptionLabel = tk.Label(frame, text = "Description:", font = ("Comic Sans MS", 12))
    descriptionLabel.place(x = 250, y = 100, anchor = tk.W)
    
    descriptionEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    descriptionEntry.place(x = 380, y = 90)
    
    costLabel = tk.Label(frame, text = "Cost:", font = ("Comic Sans MS", 12))
    costLabel.place(x = 250, y = 135, anchor = tk.W)
    
    costEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    costEntry.place(x = 380, y = 125)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backOwner(root))
    backButton.place(x = 400, y = 400, anchor = tk.CENTER)
    
    addButton = tk.Button(frame, text = "Add", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                         command = lambda : addPropertyButton(nameEntry.get(), streetEntry.get(), cityEntry.get(),
                                                             menuTextType.get(), zipEntry.get(), nearEntry.get(),
                                                             distanceEntry.get(), capacityEntry.get(), descriptionEntry.get(),
                                                             costEntry.get()))
    addButton.place(x = 100, y = 400, anchor = tk.CENTER)
    
    frame.pack()
    
def addPropertyButton(name, street, city, state, zipCode, near, distance, capacity, description, cost):
    cursor = connection.cursor()
    try:
        if (name != "" and description != "" and capacity != "" and cost != ""
           and street != "" and city != "" and state != "" and zipCode != ""
           and near != "" and distance != ""):
            cursor.callproc('add_property', [name, accountEmail[0], description, capacity, cost,
                                     street, city, state, zipCode, near, distance])
            print("add complete")
        else:
            print("invalid parameters")
    except:
        print("invalid")
    cursor.close()

def removeProperty(root):
    root.title("Remove Property")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "Remove Property", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    cursor = connection.cursor()
    statement = "select Property_Name, Descr, Capacity, Cost, " +\
        "concat(Street, ', ', City, ', ', State, ' ', Zip) from " + \
        "Property where Owner_Email = " + "'" + accountEmail[0] + "'"
    cursor.execute(statement)
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["Name", "Description", "Capacity", "Cost", "Address"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 15)
    tree.place(y = 100)
    
    displayTable(frame, tree, df, columns, 96)
    
    removeButton = tk.Button(frame, text = "Remove", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                   command = lambda : removePropertyButton(root, data, tree, tree.selection(), statement))
    removeButton.place(x = 187.5, y = 460, anchor = tk.CENTER)

    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                command = lambda : backOwner(root))
    backButton.place(x = 312.5, y = 460, anchor = tk.CENTER)
    
    frame.pack()
    
def removePropertyButton(root, data, tree, treeSelection, statement):
    name = tree.item(treeSelection)["values"][0]
    
    cursor = connection.cursor()
    cursor.callproc('remove_property', [name, accountEmail[0], currentDate[0]])
    cursor.close()
                                        
    data.clear()
    cursor = connection.cursor()
    cursor.execute(statement)
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    removeProperty(root)
    
def deleteOwner(root):
    root.title("Delete Account")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "Delete Account", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    sublabel = tk.Label(frame, text = "Are you sure you want to delete\nyour account?", font = ("Comic Sans MS", 16))
    sublabel.place(x = 250, y = 170, anchor = tk.CENTER)
    
    deleteButton = tk.Button(frame, text = "Delete", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                  command = lambda : deleteAccountButton(root))
    deleteButton.place(x = 200, y = 250, anchor = tk.CENTER)
    
    back = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                  command = lambda : backOwner(root))
    back.place(x = 300, y = 250, anchor = tk.CENTER)
    
    frame.pack()

def deleteAccountButton(root):
    cursor = connection.cursor()
    cursor.callproc('remove_owner', [accountEmail[0]])
    cursor.close()
    logout(root)
    
def rateCustomer(root):
    root.title("Rate Customer")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "Rate Customer", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    scoreLabel = tk.Label(frame, text = "Rating:", font = ("Comic Sans MS", 12))
    scoreLabel.place(x = 90, y = 460, anchor = tk.CENTER)
    
    scoreEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    scoreEntry.place(x = 120, y = 450)
    
    cursor = connection.cursor()
    statement = "select Start_Date, Customer, Property_Name, " + \
        "concat(Street, ', ', City, ', ', State, ' ', Zip) from Reserve natural join Property where Owner_Email = " + \
        "'" + accountEmail[0] + "'"
    cursor.execute(statement)
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["Reservation Date", "Customer", "Property Name", "Address"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 15)
    tree.place(y = 100)
    
    displayTable(frame, tree, df, columns, 120)
    
    rateButton = tk.Button(frame, text = "Rate", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                   command = lambda : ratePropertyButton(tree, tree.selection(), scoreEntry.get()))
    rateButton.place(x = 280, y = 460, anchor = tk.CENTER)

    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                command = lambda : backOwner(root))
    backButton.place(x = 360, y = 460, anchor = tk.CENTER)
    
    frame.pack()
    
def ratePropertyButton(tree, treeSelection, score):
    customer = tree.item(treeSelection)["values"][1]
    
    cursor = connection.cursor()
    try:
        if (score != ""):
            cursor.callproc('owner_rates_customer', [accountEmail[0], customer, score, currentDate[0]])
        else:
            print("invalid parameters")
    except:
        print("invalid")
    cursor.close()

In [268]:
### REGISTER ###
def register(root):
    root.title("Register")
    destroyPage()

    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    # basic info
    
    titleLabel = tk.Label(frame, text = "Register", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    fnameLabel = tk.Label(frame, text = "First Name:", font = ("Comic Sans MS", 12))
    fnameLabel.place(x = 0, y = 100, anchor = tk.W)
    
    fnameEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    fnameEntry.place(x = 130, y = 90)
    
    lnameLabel = tk.Label(frame, text = "Last Name:", font = ("Comic Sans MS", 12))
    lnameLabel.place(x = 0, y = 135, anchor = tk.W)
    
    lnameEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    lnameEntry.place(x = 130, y = 125)
    
    emailLabel = tk.Label(frame, text = "Email:", font = ("Comic Sans MS", 12))
    emailLabel.place(x = 0, y = 170, anchor = tk.W)
    
    emailEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    emailEntry.place(x = 130, y = 160)
    
    passwordLabel = tk.Label(frame, text = "Password:", font = ("Comic Sans MS", 12))
    passwordLabel.place(x = 0, y = 205, anchor = tk.W)
    
    passwordEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    passwordEntry.place(x = 130, y = 195)
    
    confirmLabel = tk.Label(frame, text = "Confirm:", font = ("Comic Sans MS", 12))
    confirmLabel.place(x = 0, y = 240, anchor = tk.W)
    
    confirmEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    confirmEntry.place(x = 130, y = 230)
    
    phoneLabel = tk.Label(frame, text = "Phone number:", font = ("Comic Sans MS", 12))
    phoneLabel.place(x = 0, y = 275, anchor = tk.W)
    
    phoneEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    phoneEntry.place(x = 130, y = 265)
    
    register = tk.Button(frame, text = "Register", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7)
    register.place(x = 375, y = 400, anchor = tk.CENTER)
    
    back = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                  command = lambda : logout(root))
    back.place(x = 375, y = 435, anchor = tk.CENTER)

    # cutomer info
    cardNumLabel = tk.Label(frame, text = "Card Number:", font = ("Comic Sans MS", 12))
    cardNumEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    cvvLabel = tk.Label(frame, text = "CVV:", font = ("Comic Sans MS", 12))
    cvvEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    expLabel = tk.Label(frame, text = "Exp:", font = ("Comic Sans MS", 12))
    expEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    
    # button
    
    customerButton = tk.Button(frame, text = "Customer", font = ("Comic Sans MS", 10), fg = "white", bg = "grey", width = 30,
                                  command = lambda : registerCustomer(customerButton, ownerButton, register,
                                                                      fnameEntry, lnameEntry, emailEntry,
                                                                      passwordEntry, confirmEntry, phoneEntry,
                                                                     cardNumLabel, cardNumEntry, cvvLabel,
                                                                      cvvEntry, expLabel, expEntry))
    customerButton.place(x = 125, y = 330, anchor = tk.CENTER)
    
    ownerButton = tk.Button(frame, text = "Owner", font = ("Comic Sans MS", 10), fg = "white", bg = "grey", width = 30,
                                  command = lambda : registerOwner(customerButton, ownerButton, register,
                                                                      fnameEntry, lnameEntry, emailEntry,
                                                                      passwordEntry, confirmEntry, phoneEntry,
                                                                     cardNumLabel, cardNumEntry, cvvLabel,
                                                                      cvvEntry, expLabel, expEntry))
    ownerButton.place(x = 375, y = 330, anchor = tk.CENTER)
    
    frame.pack()

def registerCustomer(customerButton, ownerButton, register,
                     fnameEntry, lnameEntry, emailEntry,
                     passwordEntry, confirmEntry, phoneEntry,
                     cardNumLabel, cardNumEntry, cvvLabel,
                     cvvEntry, expLabel, expEntry):
    customerButton.configure(bg = "black")
    ownerButton.configure(bg = "grey")
    register.configure(command = lambda : registerCustomerButton(emailEntry.get(), fnameEntry.get(), lnameEntry.get(),
                                                                passwordEntry.get(), confirmEntry.get(), phoneEntry.get(),
                                                                cardNumEntry.get(), cvvEntry.get(), expEntry.get()))
    
    global registerCondition
    
    if (registerCondition == 0):
        cardNumLabel.place(x = 0, y = 375, anchor = tk.W)
        cardNumEntry.place(x = 130, y = 365)
        cvvLabel.place(x = 0, y = 410, anchor = tk.W)
        cvvEntry.place(x = 130, y = 400)
        expLabel.place(x = 0, y = 445, anchor = tk.W)
        expEntry.place(x = 130, y = 435)
    registerCondition = 1
    
def registerOwner(customerButton, ownerButton, register,
                     fnameEntry, lnameEntry, emailEntry,
                     passwordEntry, confirmEntry, phoneEntry,
                     cardNumLabel, cardNumEntry, cvvLabel,
                     cvvEntry, expLabel, expEntry):
    customerButton.configure(bg = "grey")
    ownerButton.configure(bg = "black")
    register.configure(command = lambda : registerOwnerButton(emailEntry.get(), fnameEntry.get(), lnameEntry.get(),
                                                                passwordEntry.get(), confirmEntry.get(), phoneEntry.get()))
    
    global registerCondition
    
    if (registerCondition == 1):
        cardNumLabel.place(x = -100, y = -100, anchor = tk.W)
        cardNumEntry.place(x = -100, y = -100)
        cvvLabel.place(x = -100, y = -100, anchor = tk.W)
        cvvEntry.place(x = -100, y = -100)
        expLabel.place(x = -100, y = -100, anchor = tk.W)
        expEntry.place(x = -100, y = -100)
        
    registerCondition = 0

    
def registerCustomerButton(email, fname, lname, password, confirm, phone, card, cvv, exp):
    if (password == confirm and password != "" and fname != "" and lname != "" and email != "" and
       phone != "" and card != "" and cvv != "" and exp != ""):
        try:
            cursor = connection.cursor()
            cursor.callproc('register_customer', [email, fname, lname, password, phone, card, cvv, exp, ""])
            cursor.close()
            print("register complete")
        except:
            print("invalid")
    else:
        print("invalid parameters")
    
def registerOwnerButton(email, fname, lname, password, confirm, phone):
    if (password == confirm and password != "" and fname != "" and lname != "" and email != "" and phone != ""):
        try:
            cursor = connection.cursor()
            cursor.callproc('register_owner', [email, fname, lname, password, phone])
            cursor.close()
            print("register complete")
        except:
            print("invalid")
    else:
        print("invalid parameters")

In [269]:
### FRAME SETUP ###

def loginSetUp(root):
        root.title("Login")
        frame = tk.Frame(root, height = 500, width = 500)
        frameList.append(frame)

        logLabel = tk.Label(frame, text = "Login", font = ("Comic Sans MS", 20))
        logLabel.place(x = 250, y = 100, anchor = tk.CENTER)

        emailLabel = tk.Label(frame, text = "Email:", font = ("Comic Sans MS", 14))
        emailLabel.place(x = 170, y = 170, anchor = tk.E)

        passLabel = tk.Label(frame, text = "Password:", font = ("Comic Sans MS", 14))
        passLabel.place(x = 170, y = 220, anchor = tk.E)

        emailEntry = tk.Entry(frame, font = ("Comic Sans MS", 14), width = 20)
        emailEntry.place(x = 180, y = 155)

        passEntry = tk.Entry(frame, font = ("Comic Sans MS", 14), width = 20)
        passEntry.place(x = 180, y = 205)

        loginButton = tk.Button(frame, text = "Login", font = ("Comic Sans MS", 12), fg = "white", bg = "black", width = 8,
                               command = lambda : login(root, emailEntry.get(), passEntry.get()))
        loginButton.place(x = 250, y = 280, anchor = tk.CENTER)

        registerLabel = tk.Label(frame, text = "Not signed up?", font = ("Comic Sans MS", 10))
        registerLabel.place(x = 250, y = 320, anchor = tk.CENTER)

        registerButton = tk.Button(frame, text = "Register", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                  command = lambda : register(root))
        registerButton.place(x = 250, y = 350, anchor = tk.CENTER)

        frame.pack()

def adminHomeSetUp(root):
    root.title("Admin Home")
        
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)

    homeLabel = tk.Label(frame, text = "Admin Home", font = ("Comic Sans MS", 20))
    homeLabel.place(x = 250, y = 100, anchor = tk.CENTER)

    scheduleFlightButton = tk.Button(frame, text = "Schedule\nFlight", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8, 
                                command = lambda : scheduleFlight(root))
    scheduleFlightButton.place(x = 125, y = 180, anchor = tk.CENTER)

    removeFlightButton = tk.Button(frame, text = "Remove\nFlight", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8, 
                                   command = lambda : removeFlight(root, None, {'airline_name': '', 'flight_num': '', 'flight_date': ''}))
    removeFlightButton.place(x = 250, y = 180, anchor = tk.CENTER)

    processDateButton = tk.Button(frame, text = "Process\nDate", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                 command = lambda : processDate(root))
    processDateButton.place(x = 375, y = 180, anchor = tk.CENTER)

    viewAirportsButton = tk.Button(frame, text = "View\nAirports", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                  command = lambda : viewAirports(root, None, {'Airport_ID': '', 'Airport_Name': '', 'Time_Zone': ''}))
    viewAirportsButton.place(x = 187.5, y = 270, anchor = tk.CENTER)

    viewAirlinesButton = tk.Button(frame, text = "View\nAirlines", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                  command = lambda : viewAirlines(root, None, {'rating': '', 'total_flights': '', 'min_flight_cost': ''}))
    viewAirlinesButton.place(x = 312.5, y = 270, anchor = tk.CENTER)

    viewCustomersButton = tk.Button(frame, text = "View\nCustomers", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                   command = lambda : viewCustomers(root, None))
    viewCustomersButton.place(x = 187.5, y = 360, anchor = tk.CENTER)

    viewOwnersButton = tk.Button(frame, text = "View\nOwners", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                command = lambda : viewOwners(root, None, {'owner_name': '', 'num_properties_owned': '', 'avg_property_rating': ''}))
    viewOwnersButton.place(x = 312.5, y = 360, anchor = tk.CENTER)
    
    logoutButton = tk.Button(frame, text = "Logout", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : logout(root))
    logoutButton.place(x = 250, y = 450, anchor = tk.CENTER)

    frame.pack()
    
def customerHomeSetUp(root):
    root.title("Customer Home")
    
    if (len(frameList) != 0):
        destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)

    homeLabel = tk.Label(frame, text = "Customer Home", font = ("Comic Sans MS", 20))
    homeLabel.place(x = 250, y = 100, anchor = tk.CENTER)

    bookFlightButton = tk.Button(frame, text = "Book\nFlight", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : bookFlight(root, None, {'Airline_Name':'', 'Flight_Num':''}))
    bookFlightButton.place(x = 187.5, y = 180, anchor = tk.CENTER)

    cancelFlightButton = tk.Button(frame, text = "Cancel\nFlight", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : cancelBookedFlight(root, None))
    cancelFlightButton.place(x = 312.5, y = 180, anchor = tk.CENTER)

    viewPropertiesButton = tk.Button(frame, text = "View\nProperties", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : viewProperties(root, None))
    viewPropertiesButton.place(x = 125, y = 250, anchor = tk.CENTER)

    reservePropertyButton = tk.Button(frame, text = "Reserve\nProperty", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : reserveProperty(root, None))
    reservePropertyButton.place(x = 250, y = 250, anchor = tk.CENTER)

    cancelReservationButton = tk.Button(frame, text = "Cancel\nReservation", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8, 
                            command = lambda : cancelReservation(root))
    cancelReservationButton.place(x = 375, y = 250, anchor = tk.CENTER)

    reviewPropertyButton = tk.Button(frame, text = "Review\nProperty", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : reviewProperty(root))
    reviewPropertyButton.place(x = 125, y = 320, anchor = tk.CENTER)

    viewReservationButton = tk.Button(frame, text = "View\nReservation", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : viewReservations(root, None))
    viewReservationButton.place(x = 250, y = 320, anchor = tk.CENTER)
    
    rateOwnerButton = tk.Button(frame, text = "Rate\nOwner", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : rateOwner(root))
    rateOwnerButton.place(x = 375, y = 320, anchor = tk.CENTER)
    
    logoutButton = tk.Button(frame, text = "Logout", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : logout(root))
    logoutButton.place(x = 250, y = 390, anchor = tk.CENTER)

    frame.pack()

def ownerHomeSetUp(root):
    root.title("Owner Home")
    
    if (len(frameList) != 0):
        destroyPage()
        
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)

    homeLabel = tk.Label(frame, text = "Owner Home", font = ("Comic Sans MS", 20))
    homeLabel.place(x = 250, y = 100, anchor = tk.CENTER)

    addButton = tk.Button(frame, text = "Add\nProperty", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : addProperty(root))
    addButton.place(x = 187.5, y = 180, anchor = tk.CENTER)

    removeButton = tk.Button(frame, text = "Remove\nProperty", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : removeProperty(root))
    removeButton.place(x = 312.5, y = 180, anchor = tk.CENTER)

    rateButton = tk.Button(frame, text = "Rate\nCustomer", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : rateCustomer(root))
    rateButton.place(x = 187.5, y = 250, anchor = tk.CENTER)

    deleteButton = tk.Button(frame, text = "Delete\nAccount", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : deleteOwner(root))
    deleteButton.place(x = 312.5, y = 250, anchor = tk.CENTER)
    
    logoutButton = tk.Button(frame, text = "Logout", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : logout(root))
    logoutButton.place(x = 250, y = 390, anchor = tk.CENTER)

    frame.pack()  
    
def ownerCustomerHomeSetUp(root):
    root.title("Selection Page")
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)

    homeLabel = tk.Label(frame, text = "Login as Customer or Owner", font = ("Comic Sans MS", 20))
    homeLabel.place(x = 250, y = 100, anchor = tk.CENTER)

    customerButton = tk.Button(frame, text = "Customer", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : customerHomeSetUp(root))
    customerButton.place(x = 187.5, y = 180, anchor = tk.CENTER)

    ownerButton = tk.Button(frame, text = "Owner", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : ownerHomeSetUp(root))
    ownerButton.place(x = 312.5, y = 180, anchor = tk.CENTER)
    
    logoutButton = tk.Button(frame, text = "Logout", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : logout(root))
    logoutButton.place(x = 250, y = 390, anchor = tk.CENTER)

    frame.pack()  

In [270]:
### HELPER FUNCTIONS ###

def displayTable(frame, tree, df, columns, size):
    tree["column"] = columns
    tree["show"] = "headings"
    
    scroll = ttk.Scrollbar(orient = "vertical", command = tree.yview)
    tree.configure(yscrollcommand = scroll.set)
    scroll.place(x = 483, y = 100, height = 400)
    
    col = 0
    for column in tree["column"]:
        tree.heading(column, text = column)
        if type(size) == int:
            tree.column(column, width = size)
        else:
            tree.column(column, width = size[col])
            col += 1

    df_rows = df.to_numpy().tolist()
    ind = 0
    for row in df_rows:
        tree.insert("", "end", ind, values = row)
        ind += 1

def displayTableScroll(frame, tree, df, columns, size, scrollY, scrollSize):
    tree["column"] = columns
    tree["show"] = "headings"
    
    scroll = ttk.Scrollbar(orient = "vertical", command = tree.yview)
    tree.configure(yscrollcommand = scroll.set)
    scroll.place(x = 483, y = scrollY, height = scrollSize)
    
    col = 0
    for column in tree["column"]:
        tree.heading(column, text = column)
        if type(size) == int:
            tree.column(column, width = size)
        else:
            tree.column(column, width = size[col])
            col += 1

    df_rows = df.to_numpy().tolist()
    ind = 0
    for row in df_rows:
        tree.insert("", "end", ind, values = row)
        ind += 1
        
def destroyPage():
    frameList[0].pack_forget()
    del frameList[0]

In [271]:
def viewFlights(root, inputData):
    root.title("View Flights")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Flights", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    data = []
    if inputData == None:
        cursor = connection.cursor()
        selectStatement = "select f.flight_num, f.airline_name, f.from_airport, " + \
            "f.to_airport, f.departure_time, f.arrival_time, f.flight_date, " + \
            "calc_seats_remaining(f.flight_num, f.airline_name), f.cost, f.cost * coalesce(b.num_seats, 0) " + \
            "from flight f, book b where f.airline_name = b.airline_name and f.flight_num = b.flight_num"
        cursor.execute(selectStatement)
    
        for row in cursor.fetchall():
            data.append(row)
        cursor.close()
    else:
        data = inputData
    
    columns = ["ID", "Airline", "From", "To", "Dept. Time", "Arr. Time", "Date", "Avail. Seats", "Cost per Seat", "Total Spent"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 12)
    tree.place(y = 100)
    
    displayTable(frame, tree, df, columns, 48)

    seatLabel = tk.Label(frame, text = "Available Seats:", font = ("Comic Sans MS", 12))
    seatLabel.place(x = 180, y = 470, anchor = tk.CENTER)

    seatEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    seatEntry.place(x = 250, y = 450)

    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterBySeatsButton(root, seatEntry.get()))
    filterButton.place(x = 450, y = 450, anchor = tk.CENTER)


    frame.pack()

def filterBySeatsButton(root, seats):
    if (seats != None and int(seats) > 0):
        cursor = connection.cursor()
        selectStatement = "select f.flight_num, f.airline_name, f.from_airport, f.to_airport, f.departure_time, f.arrival_time, f.flight_date, (calc_seats_remaining(f.flight_num, f.airline_name)) as seats, f.cost, f.cost * coalesce(b.num_seats, 0) from flight f, book b where f.airline_name = b.airline_name and f.flight_num = b.flight_num and (calc_seats_remaining(f.flight_num, f.airline_name)) >= %(numSeats)s"
        cursor.execute(selectStatement, { 'numSeats': int(seats) })
        
        data = []
        for row in cursor.fetchall():
            data.append(row)
        cursor.close()
        viewFlights(root, data)
    else:
        viewFlights(root, None)

In [272]:
main()

aray@tiktok.com
